# From kaggle link: https://www.kaggle.com/datasets/msambare/fer2013

In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
trainPath = r"dataset\train"
testPath = r"dataset\test" # add the location of the saved images
folderList = os.listdir(trainPath)
folderList.sort()
print(folderList)

['angry', 'happy', 'neutral', 'sad']


In [7]:
X_train = []
X_test = []
y_train = []
y_test = []

# Load the train model to array

In [ ]:
for i, category in enumerate(folderList):
    files = os.listdir(trainPath+"/"+category)
    for file in files:
        print(category+"/"+file)
        img = cv2.imread(trainPath+"/"+category+'/{0}'.format(file),0)
        X_train.append(img)
        y_train.append(i)
print(len(X_train)) # 21005 train images

# Show the first image

In [ ]:
img1 = X_train[0]
cv2.imshow("img1", img1)
cv2.waitKey(0)

# Check the labels

In [ ]:
print(y_train)
print(len(y_train))

In [16]:
folderList = os.listdir(testPath)
folderList.sort()

In [ ]:
for i, category in enumerate(folderList):
    files = os.listdir(testPath+"/"+category)
    for file in files:
        print(category+"/"+file)
        img = cv2.imread(testPath+"/"+category+'/{0}'.format(file),0)
        X_test.append(img)
        y_test.append(i)

print("test data:")
print(len(X_test)) # 5212 test images
print(len(y_test)) #5212

# Convert the data to numpy

In [ ]:
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train)
X_test = np.array(X_test, 'float32')
y_test = np.array(y_test)

print(X_train.shape)
print(X_train[0])


In [23]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
numImages = X_train.shape[0]
X_train = X_train.reshape(numImages,48,48,1)
print(X_train[0])
print(X_train.shape)

In [ ]:
numImages = X_test.shape[0]
X_test = X_test.reshape(numImages,48,48,1)
print(X_test[0])
print(X_test.shape)

In [52]:
from tensorflow.keras.utils import to_categorical  

In [ ]:
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

print("# To categorical")
print(y_train)
print(y_train.shape)
print(y_train[0])



# Model

In [ ]:
input_shape = (48,48,1)
print(input_shape)

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape = input_shape, filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation="relu"))
model.add(Dense(4, activation="softmax"))

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
batch = 32
epochs = 30

stepsPerEpoch = np.ceil(len(X_train)/batch)
validationsPerEpoch = np.ceil(len(X_test)/batch)

stopEarly = EarlyStopping(monitor = 'val_accuracy', patience = 5)
history = model.fit(X_train, y_train, batch_size = batch, epochs = epochs, verbose = 1, validation_data = (X_test,y_test),shuffle=True, callbacks = [stopEarly])


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

In [ ]:
plt.plot(epochs, acc, 'r', label="Train accuracy")
plt.plot(epochs, val_acc, 'b', label="Train accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training and validation Accuracy")
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
plt.plot(epochs, acc, 'r', label="Train loss")
plt.plot(epochs, val_acc, 'b', label="Train loss")
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.title("Training and validation Loss")
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
modelname = r"emotionsdetection.keras"
model.save(modelname)

model = tf.keras.models.load_model("emotionsdetection.keras")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optimize model size (optional)
tflite_model = converter.convert()

# Save the converted model
with open("detectemotions.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to detectionemotions.tflite")
